## 1_agent_and_tools_basics

In [ ]:
#The core idea of agents is to use a language model to choose a sequence of actions to take. 
#In chains, a sequence of actions is hardcoded (in code). 
#In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

In [ ]:
%pip install --quiet --upgrade python-dotenv langchain langchain-community langchain-ollama

In [ ]:
import os

from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import (
    AgentExecutor,
    create_react_agent,
)
from langchain_core.tools import Tool
from langchain_ollama.llms import OllamaLLM
from langchain_ollama import ChatOllama

In [ ]:
# Load environment variables from .env file
load_dotenv()

In [ ]:
# http://api.weatherapi.com/v1/current.json?key=key&q=Lisbon

In [ ]:
from json import dumps
import requests
# Define a method that calls external api and returns json content
def get_weather(location: str) -> str:
    """Get the current weather for a specified location."""
    if not location:
        return (
            "Please provide a location and call the get_current_weather_function again."
        )
    api_params = {
        "key": os.environ.get("WEATHER_API_KEY"),
        "q": location,
        "aqi": "no",
        "alerts": "no",
    }
    response: requests.models.Response = requests.get(
        "http://api.weatherapi.com/v1/current.json", params=api_params
    )
    str_response: str = dumps(response.json())
    return str_response

In [ ]:
answer = get_weather(location="Lisbon")

In [ ]:
print(answer)

In [ ]:
from datetime import datetime

# Define a function that returns the current time
def get_current_time(*args, **kwargs):
    """Returns the current time in H:MM AM/PM format."""
    import datetime  # Import datetime module to get current time

    now = datetime.datetime.now()  # Get current time
    return now.strftime("%I:%M %p")  # Format time in H:MM AM/PM format

In [ ]:
answer = get_current_time()

In [ ]:
print(answer)

In [170]:
# List of tools available to the agent
tools = [
    Tool(
        name="Weather",  # Name of the tool
        func=get_weather,  # Function that the tool will execute
        # Description of the tool
        description="Get the current weather for a specified location",
    ),
    Tool(
        name="CurrentTime",  # Name of the tool
        func=get_current_time,  # Function that the tool will execute
        # Description of the tool
        description="Useful for when you need to know the current time",
    ),
]

In [171]:
chatbot = ChatOllama(
        base_url="http://localhost:11434",
        model="llama3.2",
        temperature=0,
)

In [172]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant which should try to answer questions as best you can.",
        ),
        ("human", "{input}"),
    ]
)

In [173]:
chain_with_tools = chatbot.bind_tools(tools)

In [174]:
chain = prompt | chain_with_tools

In [157]:
response = chain.invoke({
        "input": "Could you tell me how is the weather in Lisbon?"
})

In [158]:
print(f"Content: {response.content}")

Content: 


In [159]:
print(f"ToolCalls: {response.tool_calls}")

ToolCalls: [{'name': 'Weather', 'args': {'__arg1': 'Lisbon'}, 'id': '0e0513ac-04dd-45cc-8dd2-b79f7b0dea70', 'type': 'tool_call'}]


In [175]:
response = chain.invoke({
        "input": "Could you tell me my current time?"
})

In [161]:
print(f"Content: {response.content}")

Content: 


In [176]:
print(f"ToolCalls: {response.tool_calls}")

ToolCalls: [{'name': 'CurrentTime', 'args': {'__arg1': 'now'}, 'id': 'd3dfa918-9e21-4eb8-8395-880b107c9bf0', 'type': 'tool_call'}]


In [ ]:
#We can see that there's now no content, but there is a tool call! It wants us to call the Weather Search tool.
#This isn't calling that tool yet - it's just telling us that tool was identified and parameterized. 
#In order to actually call it, we'll want to create our agent.

In [191]:
from langchain_core.prompts.chat import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant which should try to answer questions as best you can.",
        ),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [187]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(chatbot, tools, prompt)

In [188]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [189]:
response = agent_executor.invoke({"input": "whats the weather in Lisbon?"})

In [190]:
print(f"Content: {response}")

Content: {'input': 'whats the weather in Lisbon?', 'output': 'Current weather in Lisbon is clear with a temperature of 16.2°C (61.2°F). There is no precipitation and the wind speed is 4.0 mph (6.5 km/h) from the southwest direction. The humidity is at 82% and the visibility is 10 km (6 miles).'}


In [197]:
template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

Thought: I now know the final answer

Final Answer: the final answer to the original input question

Begin!

Question: {input}

Thought:{agent_scratchpad}
"""

In [198]:
prompt = ChatPromptTemplate.from_template(template)

In [199]:
# Create the ReAct agent using the create_react_agent function
agent = create_react_agent(
    llm=chatbot,
    tools=tools,
    prompt=prompt,
    stop_sequence=True,
)

In [200]:
# Create an agent executor from the agent and tools
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [201]:
response = agent_executor.invoke({"input": "whats the weather in Lisbon?"})



> Entering new AgentExecutor chain...
Question: whats the weather in Lisbon?

Thought: I need to get the current weather for Lisbon.

Action: Weather
Action Input: Lisbon
{"location": {"name": "Lisbon", "region": "Lisboa", "country": "Portugal", "lat": 38.7167, "lon": -9.1333, "tz_id": "Europe/Lisbon", "localtime_epoch": 1733092510, "localtime": "2024-12-01 22:35"}, "current": {"last_updated_epoch": 1733092200, "last_updated": "2024-12-01 22:30", "temp_c": 16.2, "temp_f": 61.2, "is_day": 0, "condition": {"text": "Clear", "icon": "//cdn.weatherapi.com/weather/64x64/night/113.png", "code": 1000}, "wind_mph": 4.0, "wind_kph": 6.5, "wind_degree": 218, "wind_dir": "SW", "pressure_mb": 1022.0, "pressure_in": 30.18, "precip_mm": 0.0, "precip_in": 0.0, "humidity": 82, "cloud": 0, "feelslike_c": 16.2, "feelslike_f": 61.2, "windchill_c": 17.4, "windchill_f": 63.3, "heatindex_c": 17.4, "heatindex_f": 63.3, "dewpoint_c": 12.9, "dewpoint_f": 55.2, "vis_km": 10.0, "vis_miles": 6.0, "uv": 0.0, "gus

KeyboardInterrupt: 

In [ ]:
print(f"Content: {response}")